# PTAC Data EDA and Identification of Outages

In [1]:
import os
import pandas as pd
import plotly.graph_objects as go
# turn off scientific notation  in pandas
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# set max columns and rows to display
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [2]:
circuit_df = pd.read_excel('ALG_ptac_inventory.xlsx').set_index('id')
circuit_df['full_address'] = circuit_df.apply(lambda s: f"{s['address1']}_{s['city']}_{s['state']}", axis=1)

for csv in os.listdir('data'):
    if csv == 'rmsCurrent.csv':
        print(csv)

        csv_df = pd.read_csv(f'data/{csv}', 
                         parse_dates=[0], 
                         index_col=0)
        


rmsCurrent.csv


## Quick Analysis Questions

In [3]:
minutes_of_data = (~csv_df.isna()).astype(int).groupby(circuit_df['full_address'], axis=1).median().sum() / (1440*30.25)
ptac_count = circuit_df[['full_address', 'description']].drop_duplicates().groupby('full_address').count()['description']

fig = go.Figure(data=[go.Scatter(x=ptac_count, y=minutes_of_data, mode='markers+text', marker=dict(size=10, color='blue'))])
fig.update_yaxes(title='Months of Metering')
fig.update_xaxes(title='PTAC Count')
fig.update_layout(template='simple_white', title='Months of Data Metered vs PTAC Count')

In [4]:
minutes_of_data.median()

14.794329660238752

In [5]:
ptacs_df = csv_df.groupby([circuit_df['full_address'], circuit_df['description']], axis=1).sum()

In [6]:
pct_zeros = {}

for col in ptacs_df.columns:
    start = ptacs_df[col].replace(0, None).first_valid_index()
    
    if start is not None:
        valid = ptacs_df[col].loc[start:]
        pct_zeros[col] = valid[valid == 0].count() / valid.count()

pct_zeros

{('11 Sherwood Ridge Road_Brevard_NC',
  'Thru Wall Unit Corridor'): 0.8433518483446749,
 ('11 Sherwood Ridge Road_Brevard_NC',
  'Thru Wall Unit Rm 121'): 0.6760374467996788,
 ('11 Sherwood Ridge Road_Brevard_NC',
  'Thru Wall Unit Rm 122'): 0.8916575427778141,
 ('11 Sherwood Ridge Road_Brevard_NC',
  'Thru Wall Unit Rm 123'): 0.6658207868369849,
 ('11 Sherwood Ridge Road_Brevard_NC',
  'Thru Wall Unit Rm 124'): 0.9285562304903902,
 ('11 Sherwood Ridge Road_Brevard_NC',
  'Thru Wall Unit Rm 125'): 0.9128464763446551,
 ('11 Sherwood Ridge Road_Brevard_NC',
  'Thru Wall Unit Rm 126'): 0.9150891630593689,
 ('11 Sherwood Ridge Road_Brevard_NC',
  'Thru Wall Unit Rm 127'): 0.7508622246239661,
 ('11 Sherwood Ridge Road_Brevard_NC',
  'Thru Wall Unit Rm 128'): 0.8828999443147141,
 ('1328 Southeast 2nd Street_Snow Hill_NC',
  'HVAC - Rm 101'): 0.9352652631719668,
 ('1328 Southeast 2nd Street_Snow Hill_NC',
  'HVAC - Rm 103'): 0.8383928085458676,
 ('1328 Southeast 2nd Street_Snow Hill_NC',
  '

In [7]:
pct_zeros_series = pd.Series(pct_zeros)

print(pct_zeros_series.describe())

pct_zero_hist = go.Figure(data=[go.Histogram(x=pct_zeros_series)])
pct_zero_hist.update_layout(title_text='Percentage of No Current Readings in PTAC Data')
pct_zero_hist.update_xaxes(title_text='Percentage of No Current Readings', 
                           tickvals=[0, .2, .4, .6, .8, 1],
                            ticktext=['0%', '20%', '40%', '60%', '80%', '100%'])
pct_zero_hist.update_yaxes(title_text='Number of PTACs')


count   169.000
mean      0.869
std       0.077
min       0.478
25%       0.845
50%       0.892
75%       0.914
max       0.993
dtype: float64


In [8]:
print("Number of unique circuits: ", circuit_df.filter(['address1', 'description']).drop_duplicates().shape[0])
print("Number of sites: ", circuit_df.filter(['address1', 'city', 'state']).drop_duplicates().shape[0])

Number of unique circuits:  171
Number of sites:  11


In [9]:
ptacs_df = csv_df.groupby([circuit_df['full_address'], circuit_df['description']], axis=1).sum()
# ptacs_df = ptacs_df.resample('H').sum()/60

## Outage Detection

In [10]:
class RulesBasedAnomalyDetector:
    def __init__(self):
        self.temp_based_anomalies = None
        self.cohort_based_anomalies = None
        self.consistent_outages = None

    def load_data(self, all_ptacs_df, ptac_name, outside_temp):
        """_summary_

        Parameters
        ----------
        all_ptacs_df : _type_
            _description_
        ptac_name : _type_
            _description_
        outside_temp : _type_
            _description_
        """
        self.all_ptacs_df = all_ptacs_df
        self.ptac_name = ptac_name
        self.kW = self.all_ptacs_df[ptac_name]
        self.temp = outside_temp

    def set_parameters(self, 
        temp_upper_threshold=70, 
        temp_lower_threshold=50, 
        temp_kW_scaled_threshold=0.2, 
        cohort_difference_threshold=0.025, 
        consistency_n_hours_lag=12, 
        consistency_threshold=.2):
        """Set the parameters for the anomaly detection methods.

        Parameters
        ----------
        temp_upper_threshold : int
            The upper temperature threshold for detecting anomalies.
        temp_lower_threshold : int
            The lower temperature threshold for detecting anomalies.
        temp_kW_scaled_threshold : float
            The kW scaled threshold for detecting anomalies.
        cohort_difference_threshold : float
            The cohort difference threshold for detecting anomalies.
        consistency_n_hours_lag : int
            The number of hours to lag for detecting anomalies.
        consistency_threshold : float
            The consistency threshold for detecting anomalies.
        """
        self.temp_upper_threshold = temp_upper_threshold
        self.temp_lower_threshold = temp_lower_threshold
        self.temp_kW_scaled_threshold = temp_kW_scaled_threshold
        self.cohort_difference_threshold = cohort_difference_threshold
        self.consistency_n_hours_lag = consistency_n_hours_lag
        self.consistency_threshold = consistency_threshold

    def detect_temp_based_anomalies(self):
        """_summary_

        Returns
        -------
        _type_
            _description_
        """
        anomaly_index = pd.Series(index=self.kW.index, data=0)
        
        kW_scaled = (self.kW - self.kW.mean()) / self.kW.std()

        # hot and no operation
        anomaly_index[(kW_scaled < self.temp_kW_scaled_threshold) & (self.temp > self.temp_upper_threshold)] = 1

        # cold and no operation
        anomaly_index[(kW_scaled < self.temp_kW_scaled_threshold) & (self.temp < self.temp_lower_threshold)] = 1

        self.temp_based_anomalies = anomaly_index

    def detect_cohort_based_anomalies(self):
        """_summary_

        Returns
        -------
        _type_
            _description_
        """
        # scale the data
        scaled_df = self.all_ptacs_df / self.all_ptacs_df.max(0)
        avg_at_time = scaled_df.drop([self.ptac_name], axis=1).median(1)

        anomaly_index = (avg_at_time - scaled_df[self.ptac_name]) > self.cohort_difference_threshold

        self.cohort_based_anomalies = anomaly_index

    def detect_consistent_outages(self):
        """_summary_

        Returns
        -------
        _type_
            _description_
        """
        lag_df = pd.DataFrame(data={f'lag_{h}hr': self.kW.shift(h) for h in range(0, self.consistency_n_hours_lag+1)})

        anomaly_index = (lag_df.max(1) / self.kW.max()) <= self.consistency_threshold

        self.consistent_outages = anomaly_index

    def return_combined_anomalies(self):
        """_summary_

        Returns
        -------
        _type_
            _description_
        """
        return self.temp_based_anomalies & self.cohort_based_anomalies & self.consistent_outages    
    
    def anomaly_stats(self):
        """_summary_

        Returns
        -------
        _type_
            _description_
        """
        return {
            'temp_based_anomalies': self.temp_based_anomalies.sum(),
            'cohort_based_anomalies': self.cohort_based_anomalies.sum(),
            'consistent_outages': self.consistent_outages.sum(),
            'combined_anomalies': self.return_combined_anomalies().sum(),
            'percent_anomalous': self.return_combined_anomalies().sum() / self.kW.count()
        }




In [11]:
from plotly.subplots import make_subplots

stats = pd.DataFrame()

for site_address in ptacs_df.columns.get_level_values(0).unique():
    try:
        os.makedirs(f'anomalyCharts/{site_address}/')
    except:
        pass

    site_df = ptacs_df[site_address].loc['2023']
    site_df.index = pd.to_datetime(site_df.index, utc=True)
    oat_df = pd.read_csv(f'data/weather/{site_address}.csv')
    oat_df.index = pd.to_datetime(oat_df['Unnamed: 0'], utc=True)

    oat_df = oat_df.reindex(site_df.index).ffill().bfill()

    for i, ptac in enumerate(site_df.columns.get_level_values(0).unique()):
        ptac_df = site_df[ptac].to_frame()
        ptac_df = ptac_df.resample('H').mean().assign(temp=oat_df['temp'])

        detector = RulesBasedAnomalyDetector()

        detector.load_data(ptac_df, ptac, ptac_df['temp'])
        detector.set_parameters(temp_upper_threshold=70, 
                                temp_lower_threshold=50, 
                                temp_kW_scaled_threshold=0.2, 
                                cohort_difference_threshold=.05, 
                                consistency_n_hours_lag=24, 
                                consistency_threshold=.05)
        
        detector.detect_temp_based_anomalies()
        detector.detect_cohort_based_anomalies()
        detector.detect_consistent_outages()

        for k, v in detector.anomaly_stats().items():
            stats.loc[ptac, k] = v

        ptac_df['outliers'] = detector.return_combined_anomalies()

        ptac_df['outliers'] = ptac_df['outliers'].astype(bool) * ptac_df[ptac].max().max()
        
        fig = make_subplots(
            rows=1, cols=1,
            specs=[[{"secondary_y": True}]])
        
        fig.add_scatter(x=ptac_df.index, 
                                y=ptac_df[ptac],
                                name='Amps_' + ptac,
                                line=dict(color='black', width=2))

        fig.add_scatter(x=ptac_df.index,
                                y=ptac_df['outliers'],
                                name='Outliers',
                                line=dict(color='purple', width=0),
                                fill='tozeroy')

        fig.add_scatter(x=ptac_df.index,
                                y=ptac_df['temp'],
                                name='temp',
                                line=dict(color='red', width=1),
                                yaxis='y2')

        fig.update_layout(height=400, width=1000, template='simple_white')

        fig.update_yaxes(title_text='Amps', secondary_y=False)
        fig.update_yaxes(title_text='Temperature', secondary_y=True)

        fig.write_html(f'anomalyCharts/{site_address}/{ptac}.html')



In [12]:
len(ptacs_df.columns.get_level_values(0).unique())

11

In [13]:
pct_anom_hist = go.Figure([go.Histogram(x=stats.percent_anomalous)])

pct_anom_hist.update_layout(title_text='Percentage of Outage Readings in PTAC Data')
pct_anom_hist.update_xaxes(title_text='Percentage of Outage Readings', 
                           tickvals=[0, .05, .1, .15, .2, .25],
                            ticktext=['0%', '5%', '10%', '15%', '20%', '25%'])
pct_anom_hist.update_yaxes(title_text='Number of PTACs')

## Continuous Usage Detection

In [19]:
# params
usage_normalized_threshold = 0.1
min_hours_block = 48

thresh_df = (ptacs_df.abs() / ptacs_df.max(0)) > usage_normalized_threshold
blocks_df = thresh_df.diff().abs().cumsum()
match_df = blocks_df.mask(~thresh_df, 0)

blocks_df = pd.DataFrame()

for c in match_df.columns:
    blocks = pd.DataFrame(data={'len':match_df[c].groupby(match_df[c]).count(),
                                'start':match_df[c].groupby(match_df[c]).apply(lambda s: s.index[0])})
    blocks = blocks[blocks.len > min_hours_block*60]
    blocks = blocks.drop(0)

    fig = go.Figure()

    figStart = pd.to_datetime('2025-01-01')
    figEnd = pd.to_datetime('2020-01-01')

    if blocks.shape[0] > 0:
        for i, block in blocks.iterrows():
            
            block_info = {'site': c[0],
                          'ptac': c[1],
                          'length': block['len'],
                          'start': block['start'],
                          'end': block['start'] + pd.Timedelta(seconds=60*block['len'])
                          }
            
            insert_index = len(blocks_df)
            for k, v in block_info.items():
                blocks_df.loc[insert_index, k] = v

            figStart = min(figStart, block_info['start'])
            figEnd = max(figEnd, block_info['start'] + pd.Timedelta(seconds=block_info['length']))

            fig.add_shape(type='rect',
                        x0=block['start'],
                        y0=0,
                        x1=block['start'] + pd.Timedelta(seconds=60*block['len']),
                        y1=ptacs_df[block_info['start']:block_info['end']][c].max(),
                        line=dict(width=0),
                        fillcolor='red',
                        opacity=0.5)

        fig.add_scatter(x=match_df[figStart:figEnd].index, y=ptacs_df[figStart:figEnd][c])

        fig.update_layout(template='simple_white')

        fig.write_html(f'continuousUsageCharts/{c[0]}_{c[1]}.html')

In [21]:
blocks_df.to_excel('PTAC_continuous_usage_blocks.xlsx')

In [15]:
fig

In [16]:
blocks_df

,site,ptac,length,start,end
0,11 Sherwood Ridge Road_Brevard_NC,Thru Wall Unit Corridor,2971.000,2022-09-13 15:49:00,2022-09-15 17:20:00
1,11 Sherwood Ridge Road_Brevard_NC,Thru Wall Unit Corridor,3641.000,2022-11-22 00:00:00,2022-11-24 12:41:00
2,11 Sherwood Ridge Road_Brevard_NC,Thru Wall Unit Corridor,3968.000,2022-12-20 17:14:00,2022-12-23 11:22:00
3,11 Sherwood Ridge Road_Brevard_NC,Thru Wall Unit Rm 127,3303.000,2023-02-01 09:35:00,2023-02-03 16:38:00
4,11 Sherwood Ridge Road_Brevard_NC,Thru Wall Unit Rm 127,4926.000,2023-02-08 13:04:00,2023-02-11 23:10:00
5,11 Sherwood Ridge Road_Brevard_NC,Thru Wall Unit Rm 127,4428.000,2023-02-28 00:00:00,2023-03-03 01:48:00
6,11 Sherwood Ridge Road_Brevard_NC,Thru Wall Unit Rm 127,4859.000,2023-03-03 01:49:00,2023-03-06 10:48:00
7,11 Sherwood Ridge Road_Brevard_NC,Thru Wall Unit Rm 127,3387.000,2023-03-25 15:33:00,2023-03-28 00:00:00
8,11 Sherwood Ridge Road_Brevard_NC,Thru Wall Unit Rm 127,4328.000,2023-04-04 10:02:00,2023-04-07 10:10:00
9,11 Sherwood Ridge Road_Brevard_NC,Thru Wall Unit Rm 127,4391.000,2023-04-07 10:11:00,2023-04-10 11:22:00
